# Imports

In [1]:
import numpy as np
import warnings
from sklearn.model_selection import train_test_split

import pandas as pd
from pytorch_tabular import TabularModel

from pytorch_tabular.models import GANDALFConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
    ExperimentConfig
)
from pytorch_tabular import TabularModel
from pytorch_tabular.models import (
    CategoryEmbeddingModelConfig,
)
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig, ExperimentConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig
import torch
warnings.filterwarnings("ignore")
import wandb

# %load_ext autoreload
# %autoreload 2

In [2]:
wandb.login()

wandb: Currently logged in as: ddima19981998 (degt). Use `wandb login --relogin` to force relogin


True

#### Read the data

In [3]:
# read_dataset
my_df = np.load("/share/data1/GYM/full_obs_data.npz")
topo_ag_data=my_df["topo"]
dn_ag_data=my_df["dn"]
senior_ag_data=my_df["senior"]
print(topo_ag_data.shape, len(topo_ag_data))

(55772, 4296) 55772


### Topo agent 

In [4]:
topo_ag_df = pd.DataFrame(topo_ag_data, columns=[f'{i}' for i in range(topo_ag_data.shape[1])])
topo_ag_df["4295"] = topo_ag_df["4295"].astype(int).astype('category')
print(topo_ag_df.shape, topo_ag_df.shape)
print(topo_ag_df.head())

(55772, 4296) (55772, 4296)
        0    1    2    3    4    5           6           7           8  \
0  2050.0  1.0  3.0  0.0  0.0  0.0   21.900000   26.900000  360.000000   
1  2050.0  1.0  3.0  0.0  0.0  0.0   30.900000   31.500000  359.100006   
2  2050.0  1.0  3.0  0.0  0.0  0.0   42.500000   47.299999  353.899994   
3  2050.0  1.0  3.0  0.0  0.0  0.0   88.300003   88.900002  359.899994   
4  2050.0  1.0  3.0  0.0  0.0  0.0  105.199997  102.900002  359.899994   

           9  ...  4286  4287  4288  4289  4290  4291  4292  4293  4294  4295  
0  50.000000  ...  11.2   0.0   5.3   0.0   0.0   5.3   0.0   0.0   0.0     0  
1  27.400000  ...  11.2   0.0   5.3   0.0   0.0   5.3   0.0   5.7   0.0     0  
2  56.099998  ...  11.2   0.0   5.3   0.0   0.0   5.3   0.0   0.0   0.0     0  
3  81.400002  ...  11.2   0.0   5.3   0.0   0.0   5.3   0.0   5.4   0.0     0  
4  90.599998  ...  11.2   0.0   5.3   0.0   0.0   5.3   0.0   0.0   0.0     0  

[5 rows x 4296 columns]


#### Split the TOPO data

In [5]:
# Split the data
tr_val, test = train_test_split(topo_ag_df, random_state=42, test_size=0.2, shuffle=True, stratify=topo_ag_df["4295"])
train, val = train_test_split(tr_val, random_state=42, test_size=0.15, shuffle=True, stratify=tr_val["4295"])

In [6]:
data_config = DataConfig(
    target=[
        "4295"
    ],  # target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=[f'{i}' for i in range(topo_ag_data.shape[1]-1)],
    categorical_cols=[],
)
trainer_config = TrainerConfig(
    batch_size=1024,
    max_epochs=100,
    early_stopping="valid_loss",  # Monitor valid_loss for early stopping
    early_stopping_mode="min",  # Set the mode as min because for val_loss, lower is better
    early_stopping_patience=5,  # No. of epochs of degradation training will wait before terminating
    checkpoints="valid_loss",  # Save best checkpoint monitoring val_loss
    load_best=True,  # After training, load the best checkpoint
    progress_bar="none",  # Turning off Progress bar
    trainer_kwargs=dict(enable_model_summary=False),  # Turning off model summary
)
optimizer_config = OptimizerConfig()

head_config = LinearHeadConfig(
    layers="", dropout=0.1, initialization="kaiming"  # No additional layer in head, just a mapping layer to output_dim
).__dict__  # Convert to dict to pass to the model config (OmegaConf doesn't accept objects)

experiment_config = ExperimentConfig(project_name="ecml_paper", run_name="CEMC", log_target="wandb")
model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="1024-512-512",  # Number of nodes in each layer
    activation="LeakyReLU",  # Activation between each layers
    learning_rate=1e-3,
    head="LinearHead",  # Linear Head
    head_config=head_config,  # Linear Head Config
)

In [7]:
search_space = {
    "model_config__layers": ["1024-512-512", "1024-512-256", "1024-512-128"],
    "model_config.head_config__dropout": [0.1, 0.2, 0.3],
    "trainer_config__batch_size": [1024, 2048, 4096],
    "optimizer_config__optimizer": ["RAdam", "AdamW"],
}

In [8]:
from pytorch_tabular import TabularModelTuner

tuner = TabularModelTuner(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config
)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    result = tuner.tune(
        train=train,
        validation=val,
        search_space=search_space,
        strategy="grid_search",
        # cv=5, # Uncomment this to do a 5 fold cross validation 
        metric="accuracy",
        mode="max",
        progress_bar=True,
        verbose=True # Make True if you want to log metrics and params each iteration
    )

Output()

2024-05-21 18:51:55,300 - {pytorch_tabular.tabular_model_tuner:364} - INFO - Trial 1/54:                           
{'model_config.head_config__dropout': 0.1, 'model_config__layers': '1024-512-512', 'optimizer_config__optimizer':  
'RAdam', 'trainer_config__batch_size': 1024, 'loss': 0.8753462433815002, 'accuracy': 0.6835500001907349,           
'trial_id': 0} | Score: 0.6835500001907349

2024-05-21 18:52:33,430 - {pytorch_tabular.tabular_model_tuner:364} - INFO - Trial 2/54:                           
{'model_config.head_config__dropout': 0.1, 'model_config__layers': '1024-512-512', 'optimizer_config__optimizer':  
'RAdam', 'trainer_config__batch_size': 2048, 'loss': 0.862733781337738, 'accuracy': 0.6718960404396057, 'trial_id':
1} | Score: 0.6718960404396057

2024-05-21 18:53:08,834 - {pytorch_tabular.tabular_model_tuner:364} - INFO - Trial 3/54:                           
{'model_config.head_config__dropout': 0.1, 'model_config__layers': '1024-512-512', 'optimizer_config__optimizer':  
'RAdam', 'trainer_config__batch_size': 4096, 'loss': 0.8671982288360596, 'accuracy': 0.6741371750831604,           
'trial_id': 2} | Score: 0.6741371750831604

2024-05-21 18:53:57,932 - {pytorch_tabular.tabular_model_tuner:364} - INFO - Trial 4/54:                           
{'model_config.head_config__dropout': 0.1, 'model_config__layers': '1024-512-512', 'optimizer_config__optimizer':  
'AdamW', 'trainer_config__batch_size': 1024, 'loss': 0.854624330997467, 'accuracy': 0.6922157406806946, 'trial_id':
3} | Score: 0.6922157406806946

...

2024-05-21 19:27:01,154 - {pytorch_tabular.tabular_model_tuner:364} - INFO - Trial 52/54:                          
{'model_config.head_config__dropout': 0.3, 'model_config__layers': '1024-512-128', 'optimizer_config__optimizer':  
'AdamW', 'trainer_config__batch_size': 1024, 'loss': 0.8435057401657104, 'accuracy': 0.6839982271194458,           
'trial_id': 51} | Score: 0.6839982271194458

2024-05-21 19:27:53,291 - {pytorch_tabular.tabular_model_tuner:364} - INFO - Trial 53/54:                          
{'model_config.head_config__dropout': 0.3, 'model_config__layers': '1024-512-128', 'optimizer_config__optimizer':  
'AdamW', 'trainer_config__batch_size': 2048, 'loss': 0.8763486742973328, 'accuracy': 0.6777229905128479,           
'trial_id': 52} | Score: 0.6777229905128479

2024-05-21 19:28:32,374 - {pytorch_tabular.tabular_model_tuner:364} - INFO - Trial 54/54:                          
{'model_config.head_config__dropout': 0.3, 'model_config__layers': '1024-512-128', 'optimizer_config__optimizer':  
'AdamW', 'trainer_config__batch_size': 4096, 'loss': 0.8277810215950012, 'accuracy': 0.6871358156204224,           
'trial_id': 53} | Score: 0.6871358156204224

2024-05-21 19:28:32,394 - {pytorch_tabular.tabular_model_tuner:378} - INFO - Model Tuner Finished

2024-05-21 19:28:32,397 - {pytorch_tabular.tabular_model_tuner:379} - INFO - Best Score (accuracy):                
0.7005826830863953

In [9]:
pred_df_test = tabular_model.predict(test)
pred_df_val = tabular_model.predict(val)
pred_df_train = tabular_model.predict(train)

pred_df_train

NameError: name 'tabular_model' is not defined

In [ ]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score

print("Train Acc", accuracy_score(train["4095"], pred_df_train["prediction"]))
print("Train Balanced Acc", balanced_accuracy_score(train["4095"], pred_df_train["prediction"]))
print("Val Acc", accuracy_score(val["4095"], pred_df_val["prediction"]))
print("Val Balanced Acc", balanced_accuracy_score(val["4095"], pred_df_val["prediction"]))
print("Test Acc", accuracy_score(test["4095"], pred_df_test["prediction"]))
print("Balanced Test Acc", balanced_accuracy_score(test["4095"], pred_df_test["prediction"]))